# Real Estate NLQ (Natural Language Querying) Agent

This notebook shows how to make a real estate agent that supports natural language queries. It's built using the Superlinked framework and mixes vector search with agent-style logic to handle different kinds of user inputs.

The idea is pretty straightforward here: instead of making people click through filters and dropdowns, we let them just say what they want. The system takes that plain English query, figures out the intent, and routes it to the right operation—whether that's a basic search, a recommendation, a market insight, or even a comparison across cities.

Superlinked takes care of the vector-based matching, so it can handle queries like “family-friendly homes near Toronto with 2 bedrooms under \$300,000” and actually return relevant results—even if those tags aren’t explicitly in the data.

There’s also an LLM in the mix (GPT-4), which helps classify the type of query and adds reasoning when needed. That’s what brings in the agentic behavior—it’s not just matching text, it’s making decisions about *what* kind of task to run and even how to tweak the query if it’s too restrictive or vague.

## What We're Using

The dataset includes standard property info—price, number of beds and baths, full address, city, province. On top of that, there’s demographic context like population and median income, which helps when answering more nuanced questions like “Is this a good investment area?” or “What suits a young family?”

If you want to try this out with the same dataset, you can grab it here:
[Download the data](https://drive.google.com/file/d/1WJmNuq0rR5XLWQzhjtt43D8Cz0he8zyq/view?usp=sharing)

In [14]:
%%capture
!pip install pandas superlinked==28.3.1 openai python-dotenv huggingface_hub sentence-transformers

In [15]:
import pandas as pd

!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1WJmNuq0rR5XLWQzhjtt43D8Cz0he8zyq' -O data.csv

--2025-06-22 12:44:14--  https://drive.google.com/uc?export=download&id=1WJmNuq0rR5XLWQzhjtt43D8Cz0he8zyq
Resolving drive.google.com (drive.google.com)... 192.178.155.101, 192.178.155.113, 192.178.155.100, ...
Connecting to drive.google.com (drive.google.com)|192.178.155.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1WJmNuq0rR5XLWQzhjtt43D8Cz0he8zyq&export=download [following]
--2025-06-22 12:44:14--  https://drive.usercontent.google.com/download?id=1WJmNuq0rR5XLWQzhjtt43D8Cz0he8zyq&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 192.178.218.132, 2607:f8b0:4004:c25::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|192.178.218.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73732 (72K) [application/octet-stream]
Saving to: ‘data.csv’

data.csv            100%[===================>]  72.00K  --.-KB/s   

In [16]:
import os
import pandas as pd
import re
from superlinked import framework as sl
from dotenv import load_dotenv
from abc import ABC, abstractmethod
from typing import Any, Dict, List
from openai import OpenAI
from getpass import getpass

# Load environment variables
load_dotenv()

# Configure pandas display options
pd.set_option("display.max_colwidth", 500)

### Set up your OpenAI and HF Token

In [17]:
# Prompt for token (secure input)
os.environ["HUGGINGFACE_HUB_TOKEN"] = getpass("Enter your Hugging Face token: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

HUGGINGFACE_HUB_TOKEN = os.environ["HUGGINGFACE_HUB_TOKEN"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

Enter your Hugging Face token: ··········
Enter your OpenAI API key: ··········


## Abstract Tool Class

We're defining an abstract base class here that acts as a blueprint for all the tools the agent will use. Every tool will inherit from this and implement its own logic, but this makes sure they all follow the same structure.

### Tool Class

* Inherits from `ABC`, which means any subclass has to define the required methods.
* `name()`: Just returns the tool’s name—nothing fancy.
* `description()`: A short description of what the tool does.
* `use()`: The core method that does the actual work. It takes in:

  * `query`: The user's input in plain language
  * `df`: Our real estate dataset
  * `app`: The Superlinked app object to run queries
  * `master_query`: A base NLQ query we reuse
  * `openai_client`: Used when the tool needs to reason or generate something using GPT

Each custom tool will override these methods. That’s how we keep the whole thing consistent while letting each one specialize.

In [18]:
class Tool(ABC):
    @abstractmethod
    def name(self) -> str:
        pass

    @abstractmethod
    def description(self) -> str:
        pass

    @abstractmethod
    def use(self, query: str, df: pd.DataFrame, app: Any, master_query: Any, openai_client: Any) -> str:
        pass

## RealEstate Schema

This is where we define the schema for our property data using Superlinked. Think of it like a contract for what each property record should look like.

### RealEstate Class

* Inherits from `sl.Schema`, which is how Superlinked knows what fields to expect.
* The fields are pretty standard:

  * `id`: unique ID for each property
  * `city`, `address`, `province`: basic location info as strings
  * `price`, `median_family_income`: float values for anything money-related
  * `number_beds`, `number_baths`, `population`: integer fields for count-style data

Once defined, we just create an instance using `real_estate = RealEstate()`—this gets used later when setting up vector spaces and running queries.

In [19]:
class RealEstate(sl.Schema):
    id: sl.IdField
    city: sl.String
    price: sl.Float
    address: sl.String
    number_beds: sl.Integer
    number_baths: sl.Integer
    province: sl.String
    population: sl.Integer
    median_family_income: sl.Float

real_estate = RealEstate()

### Data Loading and Preprocessing

This part loads the real estate dataset (`data.csv`) and gets it into shape for the agent to use. Well just to note, I already made the `data.csv` as good as we can use for the ingestion. But I just wanted you guys to see how I did the data processing..

### What’s Happening

* We read the CSV into a pandas DataFrame and print how many records we got—just a sanity check.
* If there’s no `id` column, we generate one using the row index. The tools need unique IDs to work. As the data we are using right now, I already made sure we have the `id` there, but this is just a sanity check.
* Columns like `price`, `number_beds`, etc., are converted to the right types so we don’t run into weird bugs later (e.g., trying to run math on a string). I did this too already.
* We also pull some stats from the dataset—max values for numeric fields and the list of unique cities/provinces. These are useful later for setting up vector spaces and filtering.

In [20]:
print("Loading data...")
df = pd.read_csv("data.csv")
print(f"Loaded {len(df)} records")

# Ensure 'id' column exists
if 'id' not in df.columns:
    df['id'] = range(len(df))

# Type conversions
df['id'] = df['id'].astype(str)
df['price'] = df['price'].astype(float)
df['number_beds'] = df['number_beds'].astype(int)
df['number_baths'] = df['number_baths'].astype(int)
df['population'] = df['population'].astype(int)
df['median_family_income'] = df['median_family_income'].astype(float)

# Compute maximum values and unique categories
max_price = df["price"].max()
max_beds = df["number_beds"].max()
max_baths = df["number_baths"].max()
max_population = df["population"].max()
max_median_family_income = df["median_family_income"].max()
unique_city_categories = pd.unique(df['city']).tolist()
unique_province_categories = pd.unique(df['province']).tolist()

Loading data...
Loaded 1000 records


## Vector Space Setup

Now we’re are going to set up the vector spaces that power both the semantic and numerical search.

### This is how we are doing everything..

* `address_space` uses `all-mpnet-base-v2` to semantically match address text. So if someone says “near downtown” or “close to Queen Street,” this model helps us find similar addresses—even if the exact words don’t appear. Eassy pissy until now. Nothing fancy
* Then we’ve got numeric spaces for stuff like `price`, `beds`, `baths`, `population`, and `median_family_income`. These use min/max ranges and are set up in `MINIMUM` mode so that lower values are considered “closer” (e.g., cheaper properties are ranked higher if budget is a constraint).
* Categorical spaces like `city` and `province` handle exact matches. So if the person is asking for the property in the `Toronto`, we make sure the filteration takes consider that into account.

We also set up negative filtering so results that don’t match the requested city/province get penalized, as doing so, `relevance_score` will go down and the less relevant results will be showed in the bottom.

Finally, we combine everything into one index that’s used to run the actual search. It bundles all the schema fields and spaces into a single structure that’s ready for querying.

### Heads Up

* That `all-mpnet-base-v2` model will need internet the first time you use it.
* If your dataset changes, especially the price range, make sure to update the `max_value` settings or the scoring will be off.
* If you're working with a big dataset, embedding performance can become a bottleneck—so keep an eye on that if things start to slow down. I mean you can bring in your own `LLM` and `embedding model` of your choice.

In [21]:
print("Setting up vector spaces...")
address_space = sl.TextSimilaritySpace(text=real_estate.address, model="sentence-transformers/all-mpnet-base-v2")
price_space = sl.NumberSpace(number=real_estate.price, min_value=0, max_value=int(max_price), mode=sl.Mode.MINIMUM)
beds_space = sl.NumberSpace(number=real_estate.number_beds, min_value=0, max_value=int(max_beds), mode=sl.Mode.MINIMUM)
baths_space = sl.NumberSpace(number=real_estate.number_baths, min_value=0, max_value=int(max_baths), mode=sl.Mode.MINIMUM)
population_space = sl.NumberSpace(number=real_estate.population, min_value=0, max_value=int(max_population), mode=sl.Mode.MINIMUM)
income_space = sl.NumberSpace(number=real_estate.median_family_income, min_value=0, max_value=int(max_median_family_income), mode=sl.Mode.MINIMUM)
city_space = sl.CategoricalSimilaritySpace(category_input=real_estate.city, categories=unique_city_categories, negative_filter=-1.0, uncategorized_as_category=False)
province_space = sl.CategoricalSimilaritySpace(category_input=real_estate.province, categories=unique_province_categories, negative_filter=-1.0, uncategorized_as_category=False)

index = sl.Index(
    spaces=[address_space, price_space, beds_space, baths_space, population_space, income_space, city_space, province_space],
    fields=[real_estate.id, real_estate.city, real_estate.price, real_estate.address, real_estate.number_beds, real_estate.number_baths, real_estate.province, real_estate.population, real_estate.median_family_income]
)

Setting up vector spaces...


## OpenAI and Query Setup

Now we are just going to utilize our OpenAI client and define how our natural language queries will be processed.

* Sets up `OpenAIClientConfig` so Superlinked can use GPT to interpret queries.
* Initializes the `OpenAI` client directly too, for when tools need to generate things like insights or reformulations.
* Throws an error if the `OPEN_AI_API_KEY` isn’t set—so make sure that’s in the env before running anything. I mean general considerables right..

Then we define some query parameters—like filters for city, province, price, beds, and baths. These let us control how much flexibility or strictness we want in the results.

The actual NLQ query is configured with weighted scoring. Price gets a heavy boost (`price_weight=100.0`) because it’s usually the most important factor. Filters are applied as needed, and results are capped at 10 for now.

The key thing here is the `with_natural_query` setting—it’s what makes the system actually understand free-form text instead of needing structured input.

Just a few things to keep in mind:

* You can always tweak the weights if ranking doesn’t feel quite right. That means, if you want you can give more preference to the number of beds for a query, or maybe population can be considerable factor for you.. So if you increase the weights, it increases the relevance of that filter.


In [22]:
print("Setting up OpenAI and query...")
try:
    openai_config = sl.OpenAIClientConfig(api_key=OPENAI_API_KEY, model="gpt-4o")
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
except KeyError:
    print("ERROR: Please set your OPEN_AI_API_KEY environment variable.")
    raise

# Define query parameters
address_param = sl.Param("query_address")
city_filter_param = sl.Param("filter_by_city", options=unique_city_categories)
province_filter_param = sl.Param("filter_by_province", options=unique_province_categories)
max_price_param = sl.Param("max_price")
min_beds_param = sl.Param("min_beds")
min_baths_param = sl.Param("min_baths")

# Define NLQ query with weights
query = (
    sl.Query(
        index,
        weights={
            address_space: sl.Param("address_weight", default=1.0),
            price_space: sl.Param("price_weight", default=100.0),  # Higher weight for price
            city_space: sl.Param("city_weight", default=1.0),
            province_space: sl.Param("province_weight", default=1.0),
            beds_space: sl.Param("beds_weight", default=1.0),
            baths_space: sl.Param("baths_weight", default=1.0),
            population_space: sl.Param("population_weight", default=0.2),
            income_space: sl.Param("income_weight", default=1.0),
        }
    )
    .find(real_estate)
    .similar(address_space, address_param, sl.Param("address_similar_weight", default=1.0))
    .filter(real_estate.city == city_filter_param)
    .filter(real_estate.province == province_filter_param)
    .filter(real_estate.price <= max_price_param)
    .filter(real_estate.number_beds >= min_beds_param)
    .filter(real_estate.number_baths >= min_baths_param)
    .limit(sl.Param("limit", default=10))
    .with_natural_query(sl.Param("natural_query"), openai_config)
)

Setting up OpenAI and query...


## Data Source and Executor

Now we will use the Superlinked's in-memory ingestion, What it means is our data is loaded and processed entirely in RAM for fast operations, as having this, we can enable the faster operations without disk I/O. Ok so how it goes is...

* The data source is created using `InMemorySource`, tied to our `RealEstate` schema. It uses `DataFrameParser` to map columns from the DataFrame into the right schema fields.
* Then we initialize the `InMemoryExecutor`, which connects the data source and the vector index we built earlier.
* Once everything’s wired up, we run the executor. That gives us the `app` object, which we use to actually execute queries.
* Finally, the preprocessed DataFrame is loaded into the source so it’s ready to be queried.

If you're working with bigger datasets, in-memory probably won’t scale. As but for production, Superlinked also supports persistent vector databases so that is all good there too..


In [23]:
# Data source setup
source = sl.InMemorySource(
    real_estate,
    parser=sl.DataFrameParser(
        schema=real_estate,
        mapping={
            real_estate.id: "id",
            real_estate.city: "city",
            real_estate.price: "price",
            real_estate.address: "address",
            real_estate.number_beds: "number_beds",
            real_estate.number_baths: "number_baths",
            real_estate.province: "province",
            real_estate.population: "population",
            real_estate.median_family_income: "median_family_income"
        }
    )
)

executor = sl.InMemoryExecutor(sources=[source], indices=[index])
app = executor.run()

print(f"Ingesting {len(df)} records...")
source.put([df])
print("Data ingestion complete.")


Ingesting 1000 records...
Data ingestion complete.


## Utility Functions

I have just setted up some Utility functions, I mean just couple of helpers to make the output from the query results usable and presentable.

* `results_to_dataframe`: So it takes in the raw Superlinked query results and converts them into a pandas DataFrame. Keeps the NLQ relevance scores intact and sorts the results accordingly. If there’s nothing to show, it just returns an empty frame so nothing breaks downstream.

* `format_property_display`: Cleans up how the results are shown. If we want detailed info, it formats things like price, address, and relevance score in a more readable way (like a table). If we just need a quick list, it can handle that too. The ordering sticks to how NLQ ranked it—no reordering on our end.

These don’t touch the core logic, they’re just for better display and usability. So it's more like a post-processing logic...

In [24]:
def results_to_dataframe(query_results, original_df):
    """Convert query results to pandas DataFrame, preserving NLQ order."""
    if not query_results.entries:
        return pd.DataFrame()

    result_ids = [entry.id for entry in query_results.entries]
    scores = [entry.metadata.score for entry in query_results.entries]

    df_results = original_df[original_df['id'].astype(str).isin(result_ids)].copy()
    id_to_score = dict(zip(result_ids, scores))
    df_results['relevance_score'] = df_results['id'].astype(str).map(id_to_score)

    # Sort by relevance score
    df_results = df_results.sort_values('relevance_score', ascending=False)

    return df_results

def format_property_display(df_results, query_text, detailed=True):
    """Format property results for display, respecting NLQ order."""
    if len(df_results) == 0:
        return "No properties found."

    output = []
    if detailed:
        show_columns = ['city', 'id', 'province', 'address', 'price', 'number_beds', 'number_baths', 'relevance_score']
        df_display = df_results[show_columns].copy()
        df_display['price'] = df_display['price'].apply(lambda x: f"${x:,.0f}")
        df_display['relevance_score'] = df_display['relevance_score'].apply(lambda x: f"{x:.3f}")
        output.append(df_display.to_string(index=False))
    else:
        for idx, row in df_results.iterrows():
            output.append(f"{idx+1}. {row['city']}, {row['province']} - ${row['price']:,.0f} - {row['number_beds']}bed/{row['number_baths']}bath")

    return "\n".join(output)

## Agent Tools

Ok so here are the different tools for our agentic setup, now each one is built for a specific kind of query so the system doesn’t just throw everything into one generic search. I have described more about these tools in the blog but here is the quick snippet reference of what each one of them does..

* **PropertyRetrievalTool**: Just runs a straight-up search. It takes natural language queries like “2 bed room in Mississauga under 700K” and maps them to results using the NLQ engine.

* **PropertyRecommendationTool**: Tries to infer what the user actually wants (budget, lifestyle, family size, etc.) and then gives ranked suggestions with reasoning, pros, and cons. LLM does the heavy lifting here after we get the relevant results from the NLQ side.

* **QueryRefinementTool**: If a query gives barely any results (like too tight a budget), this tool steps in and suggests a smarter version of the query—like loosening price or location constraints. I mean we wanted to help the user as much as we can.

* **NarrativeInsightTool**: Built for more analytical queries (e.g., “what’s a good investment in Toronto?”). It pulls stats from the data and then uses the LLM to generate a readable market insight.

* **MultiStepQueryTool**: If the user asks to compare places or options, this tool breaks the query down into parts, runs them separately, and merges the results for a side-by-side view. This tool is more about how you can handle the complex queries...

Every tool leans on the same NLQ pipeline for semantic matching, and taps into OpenAI when reasoning or generation is needed.

So it's like

`NLQ Pipeline -> Gives NLQ based results (based on the relevance score) -> passed to the agent -> Final results`

In [25]:
class PropertyRetrievalTool(Tool):
    def name(self) -> str:
        return "PropertyRetrievalTool"

    def description(self) -> str:
        return "Retrieves properties based on natural language queries, leveraging NLQ intelligence for ranking"

    def use(self, query: str, df: pd.DataFrame, app: Any, master_query: Any, openai_client: Any) -> str:
        try:
            results = app.query(master_query, natural_query=query, limit=10)
            df_results = results_to_dataframe(results, df)
            if len(df_results) == 0:
                # If there are no results found, pass it to the QueryRefinementTool to get the better response
                refine_tool = QueryRefinementTool()
                return refine_tool.use(query, df, app, master_query, openai_client)
            return f"Search Results for '{query}':\n\n{format_property_display(df_results, query, detailed=True)}"
        except Exception as e:
            return f"Error in property retrieval: {str(e)}"

class PropertyRecommendationTool(Tool):
    def name(self) -> str:
        return "PropertyRecommendationTool"

    def description(self) -> str:
        return "Provides personalized property recommendations based on inferred user preferences"

    def use(self, query: str, df: pd.DataFrame, app: Any, master_query: Any, openai_client: Any) -> str:
        # Infer user profile from query
        profile_prompt = f"""
        Extract user preferences from this query, such as budget, family size, commute preferences, or lifestyle (e.g., urban, family-friendly).
        Query: "{query}"
        Return a JSON object with inferred preferences (e.g., {{"budget": 800000, "family_size": 4}}). If none, return {{}}.
        """
        try:
            profile_response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": profile_prompt}],
                temperature=0.3,
                max_tokens=100
            )
            user_profile = eval(profile_response.choices[0].message.content.strip()) or {}
        except Exception as e:
            user_profile = {}
            print(f"Error inferring user profile: {str(e)}")

        # Fetch properties using NLQ
        results = app.query(master_query, natural_query=query, limit=15)
        df_results = results_to_dataframe(results, df)
        if len(df_results) == 0:
            return "No properties found to recommend."

        # Prepare property data (top properties from NLQ ranking)
        properties_summary = [
            {
                'address': row['address'],
                'city': row['city'],
                'province': row['province'],
                'price': int(row['price']),
                'beds': int(row['number_beds']),
                'baths': int(row['number_baths']),
                'population': int(row['population']),
                'median_income': int(row['median_family_income']),
                'relevance': round(row['relevance_score'], 3)
            }
            for _, row in df_results.head(10).iterrows()
        ]

        # Generate recommendations
        prompt = f"""
        You are a real estate advisor. Provide recommendations based on:
        Query: "{query}"
        Inferred Profile: {user_profile}
        Properties (ranked by NLQ relevance): {properties_summary}

        The properties are already ranked by our intelligent NLQ system considering all factors including price relevance.
        Provide:
        1. Top 3 recommendations with reasons (include address) - respect the NLQ ranking
        2. Pros and cons for each
        3. Additional considerations (e.g., neighborhood, lifestyle fit)
        """
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=800
            )
            recommendations = response.choices[0].message.content.strip()

            # Use top properties from NLQ ranking (no additional sorting)
            formatted_properties = format_property_display(df_results.head(5), query, detailed=True)
            return f"Recommendations for '{query}':\n\n{recommendations}\n\nProperty Listings (NLQ Ranked):\n{formatted_properties}"
        except Exception as e:
            return f"Error generating recommendations: {str(e)}"

class QueryRefinementTool(Tool):
    def name(self) -> str:
        return "QueryRefinementTool"

    def description(self) -> str:
        return "Refines queries when results are sparse or irrelevant"

    def use(self, query: str, df: pd.DataFrame, app: Any, master_query: Any, openai_client: Any) -> str:
        # Try initial query
        results = app.query(master_query, natural_query=query, limit=10)
        df_results = results_to_dataframe(results, df)

        if len(df_results) >= 3:
            return f"Search Results for '{query}':\n\n{format_property_display(df_results, query, detailed=True)}"

        # Refine query if fewer than 3 results
        refine_prompt = f"""
        The query "{query}" returned {len(df_results)} results. Suggest an alternative query to get more relevant results.
        Consider relaxing filters (e.g., price, location) or expanding scope (e.g., nearby cities).
        Return a single alternative query as a string.
        """
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": refine_prompt}],
                temperature=0.3,
                max_tokens=50
            )
            new_query = response.choices[0].message.content.strip()

            # Try refined query
            results = app.query(master_query, natural_query=new_query, limit=10)
            df_new_results = results_to_dataframe(results, df)

            if len(df_new_results) == 0:
                return f"No properties found for '{query}'. Suggested query '{new_query}' also returned no results."

            return f"""
Original Query: '{query}' returned {len(df_results)} results.
Suggested Query: '{new_query}'
Results for '{new_query}' (NLQ Ranked):
{format_property_display(df_new_results, new_query, detailed=True)}
            """
        except Exception as e:
            return f"Error refining query: {str(e)}"

class NarrativeInsightTool(Tool):
    def name(self) -> str:
        return "NarrativeInsightTool"

    def description(self) -> str:
        return "Provides narrative insights for complex queries (e.g., investment potential)"

    def use(self, query: str, df: pd.DataFrame, app: Any, master_query: Any, openai_client: Any) -> str:
        results = app.query(master_query, natural_query=query, limit=10)
        df_results = results_to_dataframe(results, df)
        if len(df_results) == 0:
            return "No properties found for analysis."

        stats = {
            'avg_price': int(df_results['price'].mean()),
            'median_price': int(df_results['price'].median()),
            'price_range': (int(df_results['price'].min()), int(df_results['price'].max())),
            'avg_beds': round(df_results['number_beds'].mean(), 1),
            'avg_baths': round(df_results['number_baths'].mean(), 1),
            'cities': df_results['city'].value_counts().head(3).to_dict(),
            'avg_income': int(df_results['median_family_income'].mean())
        }

        prompt = f"""
        Provide a narrative analysis for the query: "{query}"
        Data: {stats}
        Properties (NLQ ranked by relevance): {[{k: v for k, v in row.items() if k in ['address', 'city', 'price', 'number_beds', 'number_baths', 'relevance_score']} for _, row in df_results.head(5).iterrows()]}

        Note: Properties are already intelligently ranked by our NLQ system considering price and other factors.
        Include:
        1. Market overview
        2. Investment potential (if relevant)
        3. Key considerations (e.g., location, price trends)
        """
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=600
            )
            insights = response.choices[0].message.content.strip()
            formatted_properties = format_property_display(df_results, query, detailed=True)
            return f"Insights for '{query}':\n\n{insights}\n\nProperty Listings (NLQ Ranked):\n{formatted_properties}"
        except Exception as e:
            return f"Error generating insights: {str(e)}"

class MultiStepQueryTool(Tool):
    def name(self) -> str:
        return "MultiStepQueryTool"

    def description(self) -> str:
        return "Handles queries requiring multiple NLQ calls (e.g., comparisons)"

    def use(self, query: str, df: pd.DataFrame, app: Any, master_query: Any, openai_client: Any) -> str:
        # Detect if query involves comparison
        compare_pattern = re.compile(r'\b(compare|versus|vs\.?)\b', re.IGNORECASE)
        if not compare_pattern.search(query):
            return "Query does not require comparison."

        # Extract sub-queries
        prompt = f"""
        Split the query into sub-queries for comparison.
        Query: "{query}"
        Return a JSON list of sub-queries (e.g., ["3 bedroom homes in Toronto", "3 bedroom homes in Kitchener"]).
        """
        try:
            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=100
            )
            sub_queries = eval(response.choices[0].message.content.strip())
            if not isinstance(sub_queries, list) or len(sub_queries) < 2:
                return "Unable to split query for comparison."

            # Run sub-queries (NLQ handles intelligent ranking)
            results_dict = {}
            for sub_query in sub_queries[:2]:  # Limit to 2 for simplicity
                results = app.query(master_query, natural_query=sub_query, limit=5)
                df_sub_results = results_to_dataframe(results, df)
                results_dict[sub_query] = df_sub_results

            # Generate comparison
            data_str = ""
            for sub_query, df_sub_results in results_dict.items():
                data_str += f"\n{sub_query} (NLQ ranked):\n"
                for _, row in df_sub_results.head(3).iterrows():
                    data_str += f"  - Address: {row['address']}, City: {row['city']}, Price: ${row['price']:,.0f}, Beds: {row['number_beds']}, Baths: {row['number_baths']}, Relevance: {row['relevance_score']:.3f}\n"

            compare_prompt = f"""
            Compare properties from these sub-queries (each ranked by NLQ intelligence):
            {list(results_dict.keys())}
            Data:
            {data_str}

            Note: Each property list is already ranked by our NLQ system considering all factors including price relevance.
            Provide:
            1. Side-by-side comparison (price, beds, baths, NLQ relevance)
            2. Key differences (e.g., location, value, market positioning)
            3. Recommendations for different buyer types
            """
            response = openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": compare_prompt}],
                temperature=0.7,
                max_tokens=800
            )
            comparison = response.choices[0].message.content.strip()

            # Format results
            output = [f"Comparison for '{query}':\n\n{comparison}"]
            for sub_query, df_sub_results in results_dict.items():
                if not df_sub_results.empty:
                    output.append(f"\nResults for '{sub_query}' (NLQ Ranked):\n{format_property_display(df_sub_results, sub_query, detailed=True)}")

            return "\n".join(output)
        except Exception as e:
            return f"Error processing comparison: {str(e)}"

## RealEstateAgent Class

Ok this is the main entry point—the class that actually runs the show when a query comes in. I will just run through how difefrent parts are glued..

* **Init phase**: It wires up everything—stores the dataset, the executor, the pre-built NLQ query, and the OpenAI client. It also sets up all the tools and maps them by query type so that each kind of intent goes to the right handler.

* **classify\_query**: This is where we ask the LLM to figure out *what* the user wants—basic search, a recommendation, comparison, insight, etc. If something goes wrong, it just defaults to `retrieval` to keep things safe.

* **process\_query**: Given a user query, it runs classification and then hands it off to the right tool.

In production or real life case, I think we can add logs around what category the query landed in and which tool ran—useful for debugging and understanding usage patterns.

Caching frequent classifications could help too, especially if people tend to ask the same kind of stuff over and over.

In [32]:
class RealEstateAgent:
    def __init__(self, df, app, master_query, openai_client):
        self.df = df
        self.app = app
        self.master_query = master_query
        self.openai_client = openai_client
        self.tools = [
            PropertyRetrievalTool(),
            PropertyRecommendationTool(),
            QueryRefinementTool(),
            NarrativeInsightTool(),
            MultiStepQueryTool()
        ]
        self.category_to_tool = {
            'retrieval': self.tools[0],
            'recommendation': self.tools[1],
            'refinement': self.tools[2],
            'insight': self.tools[3],
            'multistep': self.tools[4]
        }

    def classify_query(self, query: str) -> str:
        valid_categories = {'retrieval', 'recommendation', 'refinement', 'insight', 'multistep'}
        prompt = f"""
Classify the query into one category based on its intent, returning only the category name ('retrieval' if unsure):
- retrieval: Search for properties by criteria (e.g., location, price, bedrooms).
- recommendation: Personalized property suggestions for user needs.
- refinement: Strict criteria likely yielding few results, needing adjustment.
- insight: Analysis, investment advice, or market trends.
- multistep: Comparison or multiple criteria across locations.
Query: "{query}"
        """
        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=20
            )
            classification = response.choices[0].message.content.strip().lower()
            return classification if classification in valid_categories else 'retrieval'
        except Exception as e:
            print(f"Classification error: {str(e)}")
            return 'retrieval'

    def process_query(self, query: str) -> str:
        print(f"\nProcessing query: {query}")
        query_type = self.classify_query(query)
        print(f"Query classified as: {query_type}")
        tool = self.category_to_tool.get(query_type, self.tools[0])
        return tool.use(query, self.df, self.app, self.master_query, self.openai_client)

## Execution and Testing

This is where everything gets wired up and tested end-to-end.

* First, we spin up the `RealEstateAgent` instance using all the pieces defined earlier.
* Then we run a few sample queries—just to validate that each tool (retrieval, recommendation, refinement, insight, and multistep) is working as expected.

In [27]:
print("Initializing agent...")
agent = RealEstateAgent(df, app, query, openai_client)
print("Agent ready.")

# Run example queries
print("Running example queries...")
queries = [
    "3 bedroom homes in Oshawa under $600,000",
    "I want to know which will be a good option for me if I live in Toronto, most expensive one",
    "3 bedroom homes in Oshawa under $300,000",
    "What's a good investment in Toronto?",
    "Compare 3 bedroom homes in Toronto and Kitchener"
]
for q in queries:
    result = agent.process_query(q)
    print(result)
    print("\n" + "-"*60)

Initializing agent...
Agent ready.
Running example queries...

Processing query: 3 bedroom homes in Oshawa under $600,000
Query classified as: retrieval
Search Results for '3 bedroom homes in Oshawa under $600,000':

  city  id province              address    price  number_beds  number_baths relevance_score
Oshawa 171  Ontario #70 -53 TAUNTON RD E $520,000            3             3           0.418
Oshawa 603  Ontario      145 BANTING AVE $499,900            6             5           0.417

------------------------------------------------------------

Processing query: I want to know which will be a good option for me if I live in Toronto, most expensive one
Query classified as: recommendation
Recommendations for 'I want to know which will be a good option for me if I live in Toronto, most expensive one':

1. Top 3 recommendations:

   a. #LPH19 -2095 LAKE SHORE BLVD W:
   
      Reasons: This property is the most expensive on the list, priced at $4,336,900. It has 3 bedrooms and 4 ba

## Wrapping It Up

Well after everything, here we are... REAL ESTATE NLQ AGENT. Sounds cool guys.. And btw, I have wrote a complete blog for this, so if you are reading this standalone notebook, I would recommend read the blog to get better understanding.

In [36]:
natural_query = input("Enter a natural language query: ")
result = agent.process_query(natural_query)
print(result)

Enter a natural language query: 2 bedrooms and 2 bathrooms in Saint John?

Processing query: 2 bedrooms and 2 bathrooms in Saint John?
Query classified as: retrieval
Search Results for '2 bedrooms and 2 bathrooms in Saint John?':

      city  id      province               address    price  number_beds  number_baths relevance_score
Saint John 206 New Brunswick 94 Brookview Crescent $299,900            3             2           0.419
Saint John 598 New Brunswick        28 Pokiok Road $194,900            3             3           0.419
Saint John 217 New Brunswick       2121 Route  845 $875,000            3             3           0.417
